In [8]:
import cv2
import json
import numpy as np

In [9]:
class Bbox_gen:
    def __init__(self,json,image):
        self._img = self._open_image(image)
        self._annotations = self._grab_annotations(json)
        self._imagex = self._img.shape[1]
        self._imagey = self._img.shape[0]
        self._ped_instances = None
        self._grab_bboxes()
        
    def _grab_annotations(self,json_file):
        with open(json_file) as annotations:
            bbox = json.load(annotations)
        return bbox
    
    def _open_image(self,image_path):
        img = cv2.imread(image_path)        
        return img
    
    def get_annotations(self):
        return self._annotations
    
    def get_image(self):
        return self._img
    
    def get_ped_number(self):
        return self._ped_instances
    
    def _grab_bboxes(self):
        objects = self.get_annotations()['objects']
        bboxes = [object_['bbox'] for object_ in objects if object_['label']!='ignore']
        self._ped_instances = len(bboxes)
        return np.array(bboxes)
    
    def _get_coordinates(self, x_dim = None, y_dim = None, scale = False):
        coords_= self._grab_bboxes()
        
        if scale:
            x_scale = self._imagex/x_dim
            y_scale = self._imagey/y_dim
            
        for coord in coords_:
            x,y,w,h = coord
            if scale:
                coord[0] = x * x_scale
                coord[1] = y * y_scale
                coord[2] = (x + w) * x_scale
                coord[3] = (y + h ) * y_scale
            else:    
                coord[2] = x + w 
                coord[3] = y + h
        return coords_
        
    def draw_bboxes(self,image_path,x_dim = None, y_dim = None, scale = False):
        coordinates = self._get_coordinates(x_dim, y_dim, scale)
        for coord in coordinates:
            x0,y0,x1,y1 = coord
            start_point = (int(x0), int(y0))
            end_point = (int(x1), int(y1))
            image = cv2.rectangle(self._img, start_point, end_point, color=(0,255,0), thickness=2)
        if cv2.imwrite(image_path, image):
            print('Bounding boxes added successfully')
        else:
            print('There was error, bounding boxes were not added to the image')

In [10]:
img1 = Bbox_gen('Files/stuttgart_000169_000019_gtBboxCityPersons.json','Files/stuttgart_000169_000019_leftImg8bit.png')

In [4]:
img1.get_ped_number()

10

In [11]:
img1.draw_bboxes('Files/bbox_test.jpg')

Bounding boxes added successfully


In [12]:
img2 = Bbox_gen('Files/stuttgart_000169_000019_gtBboxCityPersons.json','Files/download-3.png')

In [13]:
img2.draw_bboxes('Files/modbbox_test.jpg',2048, 1024, scale = True)

Bounding boxes added successfully
